In [1]:
import re
import sys
from datetime import datetime, date, timedelta

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from termcolor import colored
from time import sleep

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

import statsmodels.api as sm
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np





#sys.stdout.reconfigure(encoding='utf-8')

teams = ["Diamondbacks", "Braves", "Orioles", "Red Sox", "Cubs", "White Sox",
            "Reds", "Guardians", "Rockies", "Tigers", "Astros", "Royals",
            "Angels", "Dodgers", "Marlins", "Brewers", "Twins", "Mets",
            "Yankees", "Athletics", "Phillies", "Pirates", "Padres", "Giants",
            "Mariners", "Cardinals", "Rays", "Rangers", "Blue Jays", "Nationals"]


### fcns

In [2]:
def check_team_names(team1, team2):
    if team1 == "D-backs" or team1 == "Arizona Diamondbacks":
        team1 = "Diamondbacks"
    elif team1 == "Atlanta Braves":
        team1 = "Braves"
    elif team1 == "Baltimore Orioles":
        team1 = "Orioles"
    elif team1 == "Boston Red Sox":
        team1 = "Red Sox"
    elif team1 == "Chicago Cubs":
        team1 = "Cubs"
    elif team1 == "Chicago White Sox":
        team1 = "White Sox"
    elif team1 == "Cincinnati Reds":
        team1 = "Reds"
    elif team1 == "Cleveland Guardians":
        team1 = "Guardians"
    elif team1 == "Colorado Rockies":
        team1 = "Rockies"
    elif team1 == "Detroit Tigers":
        team1 = "Tigers"
    elif team1 == "Houston Astros":
        team1 = "Astros"
    elif team1 == "Kansas City Royals":
        team1 = "Royals"
    elif team1 == "Los Angeles Angels":
        team1 = "Angels"
    elif team1 == "Los Angeles Dodgers":
        team1 = "Dodgers"
    elif team1 == "Miami Marlins":
        team1 = "Marlins"
    elif team1 == "Milwaukee Brewers":
        team1 = "Brewers"
    elif team1 == "Minnesota Twins":
        team1 = "Twins"
    elif team1 == "New York Mets":
        team1 = "Mets"
    elif team1 == "New York Yankees":
        team1 = "Yankees"
    elif team1 == "Philadelphia Phillies":
        team1 = "Phillies"
    elif team1 == "Pittsburgh Pirates":
        team1 = "Pirates"
    elif team1 == "San Diego Padres":
        team1 = "Padres"
    elif team1 == "San Francisco Giants":
        team1 = "Giants"
    elif team1 == "Seattle Mariners":
        team1 = "Mariners"
    elif team1 == "St. Louis Cardinals":
        team1 = "Cardinals"
    elif team1 == "Tampa Bay Rays":
        team1 = "Rays"
    elif team1 == "Texas Rangers":
        team1 = "Rangers"
    elif team1 == "Toronto Blue Jays":
        team1 = "Blue Jays"
    elif team1 == "Washington Nationals":
        team1 = "Nationals"

    if team2 == "D-backs" or team2 == "Arizona Diamondbacks":
        team2 = "Diamondbacks"
    elif team2 == "Atlanta Braves":
        team2 = "Braves"
    elif team2 == "Baltimore Orioles":
        team2 = "Orioles"
    elif team2 == "Boston Red Sox":
        team2 = "Red Sox"
    elif team2 == "Chicago Cubs":
        team2 = "Cubs"
    elif team2 == "Chicago White Sox":
        team2 = "White Sox"
    elif team2 == "Cincinnati Reds":
        team2 = "Reds"
    elif team2 == "Cleveland Guardians":
        team2 = "Guardians"
    elif team2 == "Colorado Rockies":
        team2 = "Rockies"
    elif team2 == "Detroit Tigers":
        team2 = "Tigers"
    elif team2 == "Houston Astros":
        team2 = "Astros"
    elif team2 == "Kansas City Royals":
        team2 = "Royals"
    elif team2 == "Los Angeles Angels":
        team2 = "Angels"
    elif team2 == "Los Angeles Dodgers":
        team2 = "Dodgers"
    elif team2 == "Miami Marlins":
        team2 = "Marlins"
    elif team2 == "Milwaukee Brewers":
        team2 = "Brewers"
    elif team2 == "Minnesota Twins":
        team2 = "Twins"
    elif team2 == "New York Mets":
        team2 = "Mets"
    elif team2 == "New York Yankees":
        team2 = "Yankees"
    elif team2 == "Philadelphia Phillies":
        team2 = "Phillies"
    elif team2 == "Pittsburgh Pirates":
        team2 = "Pirates"
    elif team2 == "San Diego Padres":
        team2 = "Padres"
    elif team2 == "San Francisco Giants":
        team2 = "Giants"
    elif team2 == "Seattle Mariners":
        team2 = "Mariners"
    elif team2 == "St. Louis Cardinals":
        team2 = "Cardinals"
    elif team2 == "Tampa Bay Rays":
        team2 = "Rays"
    elif team2 == "Texas Rangers":
        team2 = "Rangers"
    elif team2 == "Toronto Blue Jays":
        team2 = "Blue Jays"
    elif team2 == "Washington Nationals":
        team2 = "Nationals"

    return team1, team2

In [3]:
# this function gets all of the pitchers and their stats for todays games
def get_pitchers(teams, date):
    # URL of the StatMuse page for today's games
    formatted = date.strftime("%Y-%m-%d")
    url = f"https://www.mlb.com/probable-pitchers/{formatted}"

    # Send a GET request to the URL
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
        "Accept-Language": "en-US,en;q=0.9"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Check if the request was successful
    if response.status_code == 200:
        # print(response.status_code)
        soup = BeautifulSoup(response.content, 'html.parser')

        text_content = soup.get_text()
        cleaned_text = "\n".join(line.lstrip() for line in text_content.splitlines() if line.strip())
        # print(cleaned_text)
        lines = cleaned_text.splitlines()
        # print(lines)
        output = []
        output2 = []
        games = []

        # for teams in game:
        for i in range(len(lines) - 2):  # Stop 2 lines early to avoid index errors
            # print(lines[i])
            # print(lines[i+1])
            lines[i], lines[i+2] = check_team_names(lines[i], lines[i+2])
            relevant_lines = []
            # Check for the pattern
            if lines[i].strip() in teams and lines[i+1].strip() == '@' and lines[i+2].strip() in teams:
                # print(teams)
                team1 =lines[i].strip()
                team2 = lines[i+2].strip()
                games.append([team1, team2])
                if i + 14 < len(lines) and lines[i+4].strip() != "warmup" and lines[i+7].split()[0] != "Makeup":
                    if lines[i+4] == 'PPD':
                        continue

                    if len(lines[i+7]) > 7:
                        if lines[i+4].lower() == 'delayed':
                            relevant_lines = lines[i+5:i+11]
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                            entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]
                        elif lines[i+4].lower() == 'delay':
                            print('Delay')
                            relevant_lines = lines[i+5:i+11]
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                            entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]
                        elif lines[i+7].split()[0] == 'Follows':
                            print("here45")
                            relevant_lines = lines[i+4:i+10]
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                            entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]
                        elif lines[i+7].split()[0] == 'Moved':
                            print("Moved")
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                            entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]
                        elif lines[i+7] == 'Lou Gehrig Day':
                            print("Lou")
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                            entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]
                        elif lines[i+7] == 'Roberto Clemente Day':
                            print("Rob")
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                            entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]
                        elif lines[i+7].split()[0:4] == ['MLB', 'Little', 'League', 'Classic']:
                            print("Little League Classic")
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                            entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]
                        else:
                            print("here105")
                            relevant_lines = lines[i:i+12]
                            print(relevant_lines)
                            if lines[i+4] == 'TBD':
                                # print(lines)
                                entry1 = [team1, lines[i+3], 'A', team2, lines[i+9], lines[i+13], lines[i+14]]
                                entry2 = [team2, lines[i+9], 'H', team1, lines[i+3], lines[i+10], lines[i+11]]
                            elif lines[i+4] == 'SUSP':
                                entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                                entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]
                            else:
                                entry1 = [team1, lines[i+3], 'A', team2, lines[i+7], lines[i+11], lines[i+12]]
                                entry2 = [team2, lines[i+7], 'H', team1, lines[i+3], lines[i+8], lines[i+9]]

                        output.append([teams, relevant_lines])
                        
                    else:
                        relevant_lines = lines[i+8:i+14]
                        output.append([teams, relevant_lines])
                        
                        if lines[i+8] == 'TBD':
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+7], lines[i+9], lines[i+10]]
                        else:
                            entry1 = [team1, lines[i+3], 'A', team2, lines[i+7], lines[i+11], lines[i+12]]
                        entry2 = [team2, lines[i+7], 'H', team1, lines[i+3], lines[i+8], lines[i+9]]
                else:
                    if lines[i+4] == 'PPD':
                        continue
                    if len(lines[i+8]) > 10:
                        entry1 = [team1, lines[i+3], 'A', team2, lines[i+9], lines[i+13], lines[i+14]]
                        entry2 = [team2, lines[i+9], 'H', team1, lines[i+3], lines[i+10], lines[i+11]]
                    else:
                        entry1 = [team1, lines[i+3], 'A', team2, lines[i+8], lines[i+12], lines[i+13]]
                        entry2 = [team2, lines[i+8], 'H', team1, lines[i+3], lines[i+9], lines[i+10]]

                print(entry1)
                output2.append(entry1)
                output2.append(entry2)

        return output2, games

    else:
        print(date)
        print("probable pitchers")
        print(f"Failed to retrieve MLB webpage. Status code: {response.status_code}")



In [4]:
def get_eras(opposing_pitchers, date):
    updated_pitchers = []
    invalid_opps = []
    for p in opposing_pitchers:
        year_str = date.strftime('%Y')

        url = f"https://www.statmuse.com/mlb/ask/{p[5]}-era-{year_str}-season"

        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # print(response.status_code)
            soup = BeautifulSoup(response.content, 'html.parser')

            text_content = soup.get_text()
            lines = text_content.splitlines()
            relevant_line = lines[1]
            start_index = relevant_line.find('an ERA of')
            if start_index != -1:
                end_index = start_index + len('an ERA of') + 10
                era = relevant_line[start_index:end_index].split()[3]
            else:
                print(f"error on {p[5]}")
                print(relevant_line)
                # if p[5] == 'Luis Garcia':
                #     print("here")
                #     era = 3.35
                # else:
                new_start_index = relevant_line.find(p[5], relevant_line.find(p[5]) + len(p[5]))
                era = relevant_line[new_start_index:].split()[2 * len(p[5].split())]

            new_p = p.copy()
            new_p.append(era)
            updated_pitchers.append(new_p)
        else:
            print(f"cant find {p[5]}'s era page")
            invalid_opps.append(p[0])
            new_p = p.copy()
            new_p.append(4)
            updated_pitchers.append(new_p)

    return updated_pitchers, invalid_opps



In [5]:
import json
def fetch_mlboldds_for_date(date):
    # 1) Get the scoreboard to discover each game’s IDs and team names
    formatted = date.strftime("%Y%m%d")

    sb_url = "https://site.api.espn.com/apis/site/v2/sports/baseball/mlb/scoreboard"
    sb = requests.get(sb_url, params={"dates": formatted}).json()

    out = []
    for event in sb.get("events", []):
        comp  = event["competitions"][0]
        
        teams = {t["homeAway"]: t["team"]["name"] for t in comp["competitors"]}
        # 2) Pull the core-API odds for that specific game
        odds_url = (
            "https://sports.core.api.espn.com/v2/"
            "sports/baseball/leagues/mlb/"
            f"events/{event['id']}/competitions/{comp['id']}/odds"
        )
        r = requests.get(odds_url)
        r.raise_for_status()
        j = r.json()
        # print(json.dumps(j, indent=2))  # inspect for a 'links' or 'markets' -> 'spreads' ref

        items = j.get("items", [])
        if not items:
            continue

        # provider_with_spread = None
        # for q in items:
        #     a = q.get("awayTeamOdds", {})
        #     h = q.get("homeTeamOdds", {})
        #     if a.get("spread") is not None and a.get("spreadOdds") is not None \
        #     and h.get("spread") is not None and h.get("spreadOdds") is not None:
        #         provider_with_spread = q
        #         break

        # 3) Take the first provider’s away/home money-line
        p         = items[0]
        away_ml   = p["awayTeamOdds"]["moneyLine"]
        home_ml   = p["homeTeamOdds"]["moneyLine"]
        away_ml_str  = f"{away_ml:+d}"   # e.g. "-160" or "+105"
        home_ml_str  = f"{home_ml:+d}"   # e.g. "+135" or "-120"

        # Spread line and odds
        try:
            away_spread      = p["awayTeamOdds"]["current"]["pointSpread"]["american"]
        except:
            away_spread      = None
        try:
            away_spread_odds = p["awayTeamOdds"]["current"]["spread"]["american"]
        except:
            away_spread_odds = None
        try:
            home_spread      = p["homeTeamOdds"]["current"]["pointSpread"]["american"]
        except:
            home_spread      = None
        try:
            home_spread_odds = p["homeTeamOdds"]["current"]["spread"]["american"]
        except:
            home_spread_odds = None

        total           = p["overUnder"]
        total_over      = p["overOdds"]
        total_under     = p["underOdds"]

        total_over_str = f"{int(total_over):+d}"
        total_under_str = f"{int(total_under):+d}"
        # print(away_spread)
        # print(away_spread_odds)
        # print(home_spread)
        # print(home_spread_odds)
        # format odds with +/-
        # away_spread_str      = f"{away_spread:+.1f}" if away_spread is not None else None
        # away_spread_odds_str = f"{away_spread_odds:+d}" if away_spread_odds is not None else None
        # home_spread_str      = f"{home_spread:+.1f}" if home_spread is not None else None
        # home_spread_odds_str = f"{home_spread_odds:+d}" if home_spread_odds is not None else None
        if away_spread_odds == "EVEN":
            away_spread_odds = +100
        if home_spread_odds == "EVEN":
            home_spread_odds = +100

        out.append((
            teams["away"], teams["home"],
            away_ml_str, home_ml_str,
            away_spread, home_spread,
            away_spread_odds, home_spread_odds,
            total, total_over_str, total_under_str
        ))
        ## 0: away team,        1: home team
        ## 2: away ml,          3: home ml
        ## 4: away spread,      5: home spread
        ## 6: away spread odds, 7: home spread odds
        ## 8: total line,       9: over total odds,     10: under total odds

    return out


In [6]:
def combine_pitchers_and_odds(pitchers, odds):
    pitchers_2 = pitchers.copy()
    doubleheader = []
    spreads = []
    for p in pitchers_2:
        first_game = 0
        for matchup in odds:
            if p[0] == matchup[0]:
                if first_game == 0 and p[0] not in doubleheader:
                    first_game = 1
                    p.append(matchup[2])
                    ## 0: team, 1: ml, 2: spread, 3: spread odds, 4: total line, 5: over total odds, 6: under total odds
                    spreads.append((matchup[0], matchup[2], matchup[4], matchup[6], matchup[8], matchup[9], matchup[10]))
                elif first_game == 1 and p[0] not in doubleheader:
                    doubleheader.append(p[0])
                elif first_game == 0 and p[0] in doubleheader:
                    first_game = 1
                elif first_game == 1 and p[0] in doubleheader:
                    first_game = 1
                    p.append(matchup[2])
                    spreads.append((matchup[0], matchup[2], matchup[4], matchup[6], matchup[8], matchup[9], matchup[10]))
            elif p[0] == matchup[1]:
                if first_game == 0 and p[0] not in doubleheader:
                    first_game = 1
                    p.append(matchup[3])
                    spreads.append((matchup[1], matchup[3], matchup[5], matchup[7], matchup[8], matchup[9], matchup[10]))
                elif first_game == 1 and p[0] not in doubleheader:
                    doubleheader.append(p[0])
                elif first_game == 0 and p[0] in doubleheader:
                    first_game = 1
                elif first_game == 1 and p[0] in doubleheader:
                    first_game = 1
                    p.append(matchup[3])
                    spreads.append((matchup[1], matchup[3], matchup[5], matchup[7], matchup[8], matchup[9], matchup[10]))
    
    for p in pitchers_2:
        if len(p) == 8:
            p.append(None)


    
    return pitchers_2, doubleheader, spreads

In [7]:
def get_game_stats(matchups, date, doubleheaders):
    updated_matchups = []
    first_game = []
    for match in matchups:
        formatted = date.strftime("%m-%d-%Y")
        url = f"https://www.statmuse.com/mlb/ask?q={match[0].lower()}+vs+{match[3].lower()}+{formatted}"

        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # print(response.status_code)
            soup = BeautifulSoup(response.content, 'html.parser')
            text_content = soup.get_text()
            lines = text_content.splitlines()
            if len(lines) == 4:
                relevant_line = lines[1]
                first_index_1 = relevant_line.find(f'{match[0]}')
                first_index_2 = relevant_line.find(f'{match[0].lower()}')
                if first_index_2 < first_index_1:
                    first_index = first_index_2
                else:
                    first_index = first_index_1
                second_index = relevant_line.find(f'{match[0]}', first_index + len(f'{match[0]}'))
                stats_line = relevant_line[second_index + len(f'{match[0]}'):].split()

                if match[0] in doubleheaders and match[0] in first_game:
                    if stats_line[30] == match[0]:
                        doubleheader = match.copy()
                        doubleheader.append(stats_line[38])
                        doubleheader.append(stats_line[39])
                        doubleheader.append(stats_line[59])
                        doubleheader.append(formatted)
                        updated_matchups.append(doubleheader)
                    elif stats_line[30] + ' ' + stats_line[31] == match[0]:
                        doubleheader = match.copy()
                        doubleheader.append(stats_line[39])
                        doubleheader.append(stats_line[40])
                        doubleheader.append(stats_line[60])
                        doubleheader.append(formatted)
                        updated_matchups.append(doubleheader)
                    else:
                        doubleheader = match.copy()
                        doubleheader.append(stats_line[7])
                        doubleheader.append(stats_line[8])
                        doubleheader.append(stats_line[28])
                        doubleheader.append(formatted)
                        updated_matchups.append(doubleheader)
                else:
                    if match[0] in doubleheaders:
                        first_game.append(match[0])

                    updated_match = match.copy()
                    updated_match.append(stats_line[7])
                    updated_match.append(stats_line[8])
                    updated_match.append(stats_line[28])
                    updated_match.append(formatted)
                    updated_matchups.append(updated_match)
            else:
                print("uh oh, we made it out")
                relevant_line = lines[6]
                first_index = relevant_line.find('Team Stats')
                second_index = relevant_line.find('SLG', first_index)
                stats_line = relevant_line[first_index:second_index+10].split()
                if match[2] == 'A':
                    match.append(stats_line[6])
                    match.append(stats_line[9])
                    match.append(str(round(float(stats_line[-7]) + float(stats_line[-4]), 3)))
                    match.append(formatted)
                elif match[2] == 'H':
                    match.append(stats_line[8])
                    match.append(stats_line[11])
                    match.append(str(round(float(stats_line[-5]) + float(stats_line[-2]), 3)))
                    match.append(formatted)

        else:
            print("game stats")
            print(f"{match[0]} vs {match[3]}")
            print(date)
            print(f"Failed to retrieve MLB webpage. Status code: {response.status_code}")
            count = 0
            while response.status_code != 200:
                sleep(2)
                response = requests.get(url)
                count += 1
                if count > 10:
                    count = 0

                    break
                # Check if the request was successful
                if response.status_code == 200:
                    print(f"fixed. code: {response.status_code}")
                    soup = BeautifulSoup(response.content, 'html.parser')
                    text_content = soup.get_text()
                    lines = text_content.splitlines()
                    if len(lines) == 4:
                        relevant_line = lines[1]
                        print(relevant_line)
                        first_index_1 = relevant_line.find(f'{match[0]}')
                        first_index_2 = relevant_line.find(f'{match[0].lower()}')
                        if first_index_2 < first_index_1:
                            first_index = first_index_2
                        else:
                            first_index = first_index_1
                        second_index = relevant_line.find(f'{match[0]}', first_index + len(f'{match[0]}'))
                        stats_line = relevant_line[second_index + len(f'{match[0]}'):].split()

                        if match[0] in doubleheaders and match[0] in first_game:
                            if stats_line[30] == match[0]:
                                doubleheader = match.copy()
                                doubleheader.append(stats_line[38])
                                doubleheader.append(stats_line[39])
                                doubleheader.append(stats_line[59])
                                doubleheader.append(formatted)
                                updated_matchups.append(doubleheader)
                            elif stats_line[30] + ' ' + stats_line[31] == match[0]:
                                doubleheader = match.copy()
                                doubleheader.append(stats_line[39])
                                doubleheader.append(stats_line[40])
                                doubleheader.append(stats_line[60])
                                doubleheader.append(formatted)
                                updated_matchups.append(doubleheader)
                            else:
                                print("lets hope not")
                                print(match)
                        else:
                            if match[0] in doubleheaders:
                                first_game.append(match[0])
                            updated_match = match.copy()
                            updated_match.append(stats_line[7])
                            updated_match.append(stats_line[8])
                            updated_match.append(stats_line[28])
                            updated_match.append(formatted)
                            updated_matchups.append(updated_match)
                    else:
                        print("uh oh, we made it out")
                        relevant_line = lines[6]
                        first_index = relevant_line.find('Team Stats')
                        second_index = relevant_line.find('SLG', first_index)
                        stats_lie = relevant_line[first_index:second_index+10].split()
                        if match[2] == 'A':
                            match.append(stats_line[6])
                            match.append(stats_line[9])
                            match.append(str(round(float(stats_line[-7]) + float(stats_line[-4]), 3)))
                            match.append(formatted)
                        elif match[2] == 'H':
                            match.append(stats_line[8])
                            match.append(stats_line[11])
                            match.append(str(round(float(stats_line[-5]) + float(stats_line[-2]), 3)))
                            match.append(formatted)
                else:
                    print(f"still working. code: {response.status_code}")
                    

    return updated_matchups




### change

In [8]:
month = 9
day = 17
# 2 days ago
start_date = datetime(2025, month, day-2)
# start_date = datetime(2025, 8, 31)
# yesterday
date_to_add = datetime(2025, month, day-1)
# date_to_add = datetime(2025, 8, 31)
# today(predicting day)
date_to_predict = datetime(2025, month, day)

### daily

In [9]:
print(date_to_add.strftime("%m_%d"))

09_16


In [10]:
team_entries = pd.read_csv(f"daily_entries/daily_entries_{start_date.strftime("%m_%d")}.csv")
del team_entries['Unnamed: 0']
team_entries

,Date,Team,Win %,Score,Location,Moneyline Odds,Opponent,Opponent Win %,Opposing Pitcher ERA,Opposing Pitcher Hand,Runs,Hits,OPS,Win % #,Opponent Win % #
0,03-30-2023,Braves,(1-0),7,0,-230.0,Nationals,(0-0),5.20,1,7,12,0.741,1.000,0.000
1,03-30-2023,Nationals,(0-0),2,1,180.0,Braves,(1-0),2.55,1,2,8,0.589,0.000,1.000
2,03-30-2023,Giants,(0-1),0,0,145.0,Yankees,(0-0),2.63,0,0,4,0.345,0.000,0.000
3,03-30-2023,Yankees,(0-0),5,1,-165.0,Giants,(0-1),3.25,0,5,8,0.732,0.000,0.000
4,03-30-2023,Orioles,(1-0),10,0,105.0,Red Sox,(0-0),7.04,0,10,15,1.179,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13632,09-15-2025,Cardinals,(73-78),6,1,-115.0,Reds,(75-75),3.86,0,6,10,0.677,0.483,0.500
13633,09-15-2025,Rangers,(79-72),3,0,110.0,Astros,(82-69),4.04,0,3,10,0.696,0.523,0.543
13634,09-15-2025,Astros,(82-69),6,1,-130.0,Rangers,(79-72),3.82,0,6,6,0.597,0.543,0.523
13635,09-15-2025,Phillies,(90-61),6,0,100.0,Dodgers,(84-66),3.47,1,6,5,0.709,0.596,0.560


In [11]:
def get_for_date(date):
    date_str = date.strftime("%m-%d-%Y")

    opposing_pitchers, games = get_pitchers(teams, date)
    updated_pitchers, invalid_opps = get_eras(opposing_pitchers, date)
    odds = fetch_mlboldds_for_date(date)
    # for o in odds:
    #     print(o)
    matchups, doubles, spreads = combine_pitchers_and_odds(updated_pitchers, odds)
    # for p in spreads:
    #     print(p)
    final_matchups = get_game_stats(matchups, date, doubles)
    # print(final_matchups)
    desired_indices = [12, 0, 1, 9, 2, 8, 3, 4, 7, 6, 9, 10, 11]

    final_matchups_v2 = [[row[i] for i in desired_indices] for row in final_matchups]


    model_test_entries = pd.DataFrame(final_matchups_v2, columns=['Date', 'Team', 'Win %', 'Score', 'Location', 'Moneyline Odds', 'Opponent',
                                                    'Opponent Win %','Opposing Pitcher ERA', 'Opposing Pitcher Hand', 'Runs', 'Hits', 'OPS'])

    print(f"{len(model_test_entries)} new entries")
    copy_df = pd.concat([team_entries, model_test_entries], ignore_index=True)

    import re


    for i in range(len(copy_df)):

        me_lines = re.split(r"[-()]", copy_df.loc[i, 'Win %'])
        opp_lines = re.split(r"[-()]", copy_df.loc[i, 'Opponent Win %'])

        if len(me_lines) < 4:
            if len(opp_lines) < 4:
                continue
            else:
                opp_denom = float(opp_lines[1]) + float(opp_lines[2])

                if opp_denom == 0:
                    copy_df.loc[i,'Opponent Win % #'] = 0
                else:
                    copy_df.loc[i,'Opponent Win % #'] = round(float(opp_lines[1]) / opp_denom, 3)

        else:
            me_denom = float(me_lines[1]) + float(me_lines[2])

            if me_denom == 0:
                copy_df.loc[i,'Win % #'] = 0
            else:
                copy_df.loc[i,'Win % #'] = round(float(me_lines[1]) / me_denom, 3)

            if len(opp_lines) < 4:
                continue
            else:
                opp_denom = float(opp_lines[1]) + float(opp_lines[2])

                if opp_denom == 0:
                    copy_df.loc[i,'Opponent Win % #'] = 0
                else:
                    copy_df.loc[i,'Opponent Win % #'] = round(float(opp_lines[1]) / opp_denom, 3)

    for i in range(len(copy_df)):
        if copy_df.loc[i, 'Location'] == 'H':
            copy_df.loc[i, 'Location'] = 1
        elif copy_df.loc[i, 'Location'] == 'A':
            copy_df.loc[i, 'Location'] = 0

        if copy_df.loc[i, 'Opposing Pitcher Hand'] == 'LHP':
            copy_df.loc[i, 'Opposing Pitcher Hand'] = 1
        elif copy_df.loc[i, 'Opposing Pitcher Hand'] == 'RHP':
            copy_df.loc[i, 'Opposing Pitcher Hand'] = 0

    # team_entries

    # team_entries
    for i in range(len(copy_df)):
        if copy_df.loc[i, 'Opposing Pitcher Hand'] == '右投手':
            copy_df.loc[i, 'Opposing Pitcher Hand'] = 0
        elif copy_df.loc[i, 'Opposing Pitcher Hand'] == '左投手':
            copy_df.loc[i, 'Opposing Pitcher Hand'] = 1
        elif copy_df.loc[i, 'Opposing Pitcher Hand'] == 'LD':
            copy_df.loc[i, 'Opposing Pitcher Hand'] = 0
        elif copy_df.loc[i, 'Opposing Pitcher Hand'] == 'LZ':
            copy_df.loc[i, 'Opposing Pitcher Hand'] = 1
    return copy_df, model_test_entries, doubles


copy_df, yesterday_entries, doubles = get_for_date(date_to_add)



['Braves', '(68-83)', 'A', 'Nationals', '(62-89)', 'Jake Irvin', 'RHP']
['Guardians', '(79-71)', 'A', 'Tigers', '(85-66)', 'Casey Mize', 'RHP']
['Cubs', '(87-64)', 'A', 'Pirates', '(65-87)', 'Paul Skenes', 'RHP']
['Braves', '(69-83)', 'A', 'Nationals', '(62-90)', 'MacKenzie Gore', 'LHP']
['Athletics', '(71-80)', 'A', 'Red Sox', '(82-69)', 'Connelly Early', 'LHP']
['Padres', '(82-69)', 'A', 'Mets', '(78-73)', 'Clay Holmes', 'RHP']
['Blue Jays', '(89-62)', 'A', 'Rays', '(73-78)', 'Ryan Pepiot', 'RHP']
['Angels', '(69-82)', 'A', 'Brewers', '(92-59)', 'Freddy Peralta', 'RHP']
['Yankees', '(84-67)', 'A', 'Twins', '(66-85)', 'Zebby Matthews', 'RHP']
['Mariners', '(83-68)', 'A', 'Royals', '(75-76)', 'Michael Wacha', 'RHP']
['Orioles', '(71-80)', 'A', 'White Sox', '(57-95)', 'Shane Smith', 'RHP']
['Reds', '(75-76)', 'A', 'Cardinals', '(74-78)', 'Michael McGreevy', 'RHP']
['Rangers', '(79-73)', 'A', 'Astros', '(83-69)', 'AJ Blubaugh', 'RHP']
here105
['Marlins', '@', 'Rockies', '(71-80)', 'final

In [12]:
# copy_df.loc[11980, 'Opposing Pitcher ERA'] = 4.83
# copy_df.loc[11980, 'Opposing Pitcher Hand'] = 1
# copy_df.loc[10993, 'Opposing Pitcher Hand'] = 1
# copy_df.loc[10985, 'Opposing Pitcher Hand'] = 0
# copy_df.loc[11922, 'Opposing Pitcher Hand'] = 1
# copy_df.loc[13743, 'Opposing Pitcher Hand'] = 0
# copy_df.loc[13743, 'Opposing Pitcher ERA'] = 3.93
# copy_df.loc[13747, 'Opposing Pitcher Hand'] = 0
# copy_df.loc[13747, 'Opposing Pitcher ERA'] = 4.56


In [13]:
copy_df.tail(30)

,Date,Team,Win %,Score,Location,Moneyline Odds,Opponent,Opponent Win %,Opposing Pitcher ERA,Opposing Pitcher Hand,Runs,Hits,OPS,Win % #,Opponent Win % #
13639,09-16-2025,Guardians,(79-71),7,0,+120,Tigers,(85-66),3.91,0,7,13,.853,0.527,0.563
13640,09-16-2025,Tigers,(85-66),5,1,-145,Guardians,(79-71),3.21,1,5,9,.810,0.563,0.527
13641,09-16-2025,Cubs,(87-64),4,0,+110,Pirates,(65-87),1.97,0,4,14,.972,0.576,0.428
13642,09-16-2025,Pirates,(65-87),1,1,-130,Cubs,(87-64),2.67,0,1,4,.350,0.428,0.576
13643,09-16-2025,Braves,(69-83),6,0,-185,Nationals,(62-90),4.17,1,6,13,.943,0.454,0.408
13644,09-16-2025,Nationals,(62-90),3,1,+155,Braves,(69-83),2.63,1,3,8,.584,0.408,0.454
13645,09-16-2025,Athletics,(71-80),2,0,+135,Red Sox,(82-69),1.88,1,2,9,.583,0.470,0.543
13646,09-16-2025,Red Sox,(82-69),1,1,-160,Athletics,(71-80),4.11,1,1,8,.633,0.543,0.470
13647,09-16-2025,Padres,(82-69),3,0,+105,Mets,(78-73),3.66,0,3,7,.781,0.543,0.517
13648,09-16-2025,Mets,(78-73),8,1,-125,Padres,(82-69),3.57,0,8,10,1.058,0.517,0.543


In [14]:
copy_df.to_csv(f"daily_entries/daily_entries_{date_to_add.strftime("%m_%d")}.csv")

In [15]:
copied = copy_df.sort_values(['Team', 'Date'])


# 3) Compute L5 and L10 by grouping on Team:
copied['Runs L5'] = (
    copied
    .groupby('Team')['Runs']
    .transform(lambda x: x.shift(1)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)

copied['Hits L5'] = (
    copied
    .groupby('Team')['Hits']
    .transform(lambda x: x.shift(1)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)

copied['OPS L5'] = (
    copied
    .groupby('Team')['OPS']
    .transform(lambda x: x.shift(1)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)


copied['Runs L10'] = (
    copied
    .groupby('Team')['Runs']
    .transform(lambda x: x.shift(1)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

copied['Hits L10'] = (
    copied
    .groupby('Team')['Hits']
    .transform(lambda x: x.shift(1)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

copied['OPS L10'] = (
    copied
    .groupby('Team')['OPS']
    .transform(lambda x: x.shift(1)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

# 3) Compute L5 and L10 by grouping on Team:
copied['Next Runs L5'] = (
    copied
    .groupby('Team')['Runs']
    .transform(lambda x: x.shift(0)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)

copied['Next Hits L5'] = (
    copied
    .groupby('Team')['Hits']
    .transform(lambda x: x.shift(0)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)

copied['Next OPS L5'] = (
    copied
    .groupby('Team')['OPS']
    .transform(lambda x: x.shift(0)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)


copied['Next Runs L10'] = (
    copied
    .groupby('Team')['Runs']
    .transform(lambda x: x.shift(0)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

copied['Next Hits L10'] = (
    copied
    .groupby('Team')['Hits']
    .transform(lambda x: x.shift(0)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

copied['Next OPS L10'] = (
    copied
    .groupby('Team')['OPS']
    .transform(lambda x: x.shift(0)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)



In [16]:

df = copied[['Date', 'Team', 'Win % #', 'Score', 'Location', 'Moneyline Odds',
        'Opponent', 'Opponent Win % #',
        'Opposing Pitcher ERA', 'Opposing Pitcher Hand',
        'Runs L5', 'Hits L5', 'OPS L5',
        'Runs L10', 'Hits L10', 'OPS L10',
        'Next Runs L5', 'Next Runs L10', 'Next Hits L5',
        'Next Hits L10', 'Next OPS L5', 'Next OPS L10']]
df

numeric_cols = [
    'Win % #',
    'Score',
    'Moneyline Odds',
    'Opponent Win % #',
    'Opposing Pitcher ERA',
    'Runs L5',
    'Hits L5',
    'OPS L5',
    'Runs L10',
    'Hits L10',
    'OPS L10',
    'Next Runs L5', 'Next Runs L10', 'Next Hits L5',
    'Next Hits L10', 'Next OPS L5', 'Next OPS L10'
]

int_cols = ['Score', 'Location', 'Moneyline Odds', 'Opposing Pitcher Hand',]
float_cols = ['Win % #', 'Opponent Win % #', 'Opposing Pitcher ERA', 'Runs L5',
                'Hits L5', 'OPS L5', 'Runs L10', 'Hits L10', 'OPS L10',
                'Next Runs L5', 'Next Runs L10', 'Next Hits L5',
                'Next Hits L10', 'Next OPS L5', 'Next OPS L10']

df_clean = df.dropna(subset=numeric_cols).copy()
df_clean.reset_index(drop=True, inplace=True)

df_clean[int_cols] = df_clean[int_cols].astype(int)
df_clean[float_cols] = df_clean[float_cols].astype(float)

df_clean['Date'] = pd.to_datetime(df_clean['Date'])


df_clean['month_sin']  = np.sin(2 * np.pi * df_clean['Date'].dt.month  / 12)
df_clean['month_cos']  = np.cos(2 * np.pi * df_clean['Date'].dt.month  / 12)

df_red = pd.get_dummies(
    df_clean,
    columns=['Team', 'Opponent'],
    prefix=['Team', 'Opp'],
    drop_first=True
)

keep_cols = ['Win % #', 'Score', 'Location', 'Moneyline Odds', 'Opponent Win % #',
            'Opposing Pitcher ERA', 'Opposing Pitcher Hand', 'Runs L5', 'Hits L5',
            'OPS L5', 'Runs L10', 'Hits L10', 'OPS L10', 'month_sin', 'month_cos']
df_red = df_red[keep_cols]

### predict score

In [17]:
y = df_red['Score']
X = df_red.drop(columns=['Score'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [18]:
from lightgbm import LGBMRegressor
best_params = {'bagging_fraction': 0.6, 'bagging_freq': 5, 'feature_fraction': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200, 'num_leaves': 31}


final_lgb = LGBMRegressor(**best_params)

final_lgb.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000362 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1697
[LightGBM] [Info] Number o

LGBMRegressor(bagging_fraction=0.6, bagging_freq=5, feature_fraction=1.0,
              learning_rate=0.01, max_depth=3, n_estimators=200)

## get data to predict(today)

In [19]:
# my_date = datetime(2025, 6, 14)

def get_test(date, df_clean):
    opposing_pitchers, games = get_pitchers(teams, date)
    # print(opposing_pitchers)
    updated_pitchers, invalid_opp_pitchers = get_eras(opposing_pitchers, date)
    # print(updated_pitchers)
    odds = fetch_mlboldds_for_date(date)
    matchups, doubles, spreads = combine_pitchers_and_odds(updated_pitchers, odds)
    
    # print(f"date: {date}")
    # print(f"max date: {max(df_clean['Date'])}")
    day_ago_df = df_clean[df_clean['Date'] == (max(df_clean['Date']))]
    games = []
    for match in matchups:
        print(match)
        iter = 2
        while day_ago_df[day_ago_df['Team'] == match[0]].sort_values('Date', ascending=False).empty:
            print("got here")
            print(match)
            print(iter)
            day_ago_df = df_clean[df_clean['Date'] == (date - timedelta(days=iter))]
            iter += 1
            # continue
        match.append(float(day_ago_df[day_ago_df['Team'] == match[0]].sort_values('Date', ascending=False).iloc[0]['Next Runs L5']))
        match.append(float(day_ago_df[day_ago_df['Team'] == match[0]].sort_values('Date', ascending=False).iloc[0]['Next Runs L10']))
        match.append(float(day_ago_df[day_ago_df['Team'] == match[0]].sort_values('Date', ascending=False).iloc[0]['Next Hits L5']))
        match.append(float(day_ago_df[day_ago_df['Team'] == match[0]].sort_values('Date', ascending=False).iloc[0]['Next Hits L10']))
        match.append(float(day_ago_df[day_ago_df['Team'] == match[0]].sort_values('Date', ascending=False).iloc[0]['Next OPS L5']))
        match.append(float(day_ago_df[day_ago_df['Team'] == match[0]].sort_values('Date', ascending=False).iloc[0]['Next OPS L10']))
        match.append(date)
        # print(match)
    print(f"matchups: {len(matchups)}")

    desired_indices = [15, 0, 1, 2, 8, 3, 4, 7, 6, 9, 10, 11, 12, 13, 14]

    final_matchups = [[row[i] for i in desired_indices] for row in matchups]

    day_entries = pd.DataFrame(final_matchups, columns=['Date', 'Team', 'Win %', 'Location', 'Moneyline Odds', 'Opponent',
                                                    'Opponent Win %','Opposing Pitcher ERA', 'Opposing Pitcher Hand', 
                                                    'Runs L5', 'Runs L10', 'Hits L5', 'Hits L10', 'OPS L5', 'OPS L10'])

    print(f"day_entries: {len(day_entries)}")

    for i in range(len(day_entries)):

        me_lines = re.split(r"[-()]", day_entries.loc[i, 'Win %'])
        opp_lines = re.split(r"[-()]", day_entries.loc[i, 'Opponent Win %'])

        if len(me_lines) < 4:
            if len(opp_lines) < 4:
                continue
            else:
                opp_denom = float(opp_lines[1]) + float(opp_lines[2])

                if opp_denom == 0:
                    day_entries.loc[i,'Opponent Win % #'] = 0
                else:
                    day_entries.loc[i,'Opponent Win % #'] = round(float(opp_lines[1]) / opp_denom, 3)

        else:
            me_denom = float(me_lines[1]) + float(me_lines[2])

            if me_denom == 0:
                day_entries.loc[i,'Win % #'] = 0
            else:
                day_entries.loc[i,'Win % #'] = round(float(me_lines[1]) / me_denom, 3)

            if len(opp_lines) < 4:
                continue
            else:
                opp_denom = float(opp_lines[1]) + float(opp_lines[2])

                if opp_denom == 0:
                    day_entries.loc[i,'Opponent Win % #'] = 0
                else:
                    day_entries.loc[i,'Opponent Win % #'] = round(float(opp_lines[1]) / opp_denom, 3)

    for i in range(len(day_entries)):
        if day_entries.loc[i, 'Location'] == 'H':
            day_entries.loc[i, 'Location'] = 1
        elif day_entries.loc[i, 'Location'] == 'A':
            day_entries.loc[i, 'Location'] = 0

        if day_entries.loc[i, 'Opposing Pitcher Hand'] == 'LHP':
            day_entries.loc[i, 'Opposing Pitcher Hand'] = 1
        elif day_entries.loc[i, 'Opposing Pitcher Hand'] == 'RHP':
            day_entries.loc[i, 'Opposing Pitcher Hand'] = 0
        elif day_entries.loc[i, 'Opposing Pitcher Hand'] == '右投手':
            day_entries.loc[i, 'Opposing Pitcher Hand'] = 0
        elif day_entries.loc[i, 'Opposing Pitcher Hand'] == '左投手':
            day_entries.loc[i, 'Opposing Pitcher Hand'] = 1
        elif day_entries.loc[i, 'Opposing Pitcher Hand'] == 'LD':
            day_entries.loc[i, 'Opposing Pitcher Hand'] = 0
        elif day_entries.loc[i, 'Opposing Pitcher Hand'] == 'LZ':
            day_entries.loc[i, 'Opposing Pitcher Hand'] = 1
        else:
            day_entries.loc[i, 'Opposing Pitcher Hand'] = 0
        

    numeric_cols = [
        'Win % #',
        # 'Score',
        'Moneyline Odds',
        'Opponent Win % #',
        'Opposing Pitcher ERA',
        'Runs L5',
        'Hits L5',
        'OPS L5',
        'Runs L10',
        'Hits L10',
        'OPS L10'
    ]

    int_cols = ['Location', 'Moneyline Odds', 'Opposing Pitcher Hand',]
    float_cols = ['Win % #', 'Opponent Win % #', 'Opposing Pitcher ERA', 'Runs L5',
                    'Hits L5', 'OPS L5', 'Runs L10', 'Hits L10', 'OPS L10']

    print(f"day_entries: {len(day_entries)}")

    df_clean = day_entries.dropna(subset=numeric_cols).copy()
    df_clean.reset_index(drop=True, inplace=True)

    print(f"df_clean: {len(df_clean)}")

    df_clean[int_cols] = df_clean[int_cols].astype(int)
    df_clean[float_cols] = df_clean[float_cols].astype(float)

    df_clean['Date'] = pd.to_datetime(df_clean['Date'])


    df_clean['month_sin']  = np.sin(2 * np.pi * df_clean['Date'].dt.month  / 12)
    df_clean['month_cos']  = np.cos(2 * np.pi * df_clean['Date'].dt.month  / 12)

    games = df_clean['Team'].values


    keep_cols_day = ['Win % #', 'Location', 'Moneyline Odds', 'Opponent Win % #',
                'Opposing Pitcher ERA', 'Opposing Pitcher Hand', 'Runs L5', 'Hits L5',
                'OPS L5', 'Runs L10', 'Hits L10', 'OPS L10', 'month_sin', 'month_cos']
    df_red = df_clean[keep_cols_day]

    return df_red, games, invalid_opp_pitchers, spreads


day_df, games, invalid_opp_pitchers, spreads = get_test(date_to_predict, df_clean)
day_df


['Cubs', '(88-64)', 'A', 'Pirates', '(65-88)', 'Johan Oviedo', 'RHP']
['Reds', '(76-76)', 'A', 'Cardinals', '(74-79)', 'Andre Pallante', 'RHP']
['Orioles', '(72-80)', 'A', 'White Sox', '(57-96)', 'Martín Pérez', 'LHP']
['Giants', '(76-76)', 'A', 'Diamondbacks', '(77-76)', 'Brandon Pfaadt', 'RHP']
['Braves', '(70-83)', 'A', 'Nationals', '(62-91)', 'Brad Lord', 'RHP']
['Guardians', '(80-71)', 'A', 'Tigers', '(85-67)', 'Jack Flaherty', 'RHP']
['Athletics', '(71-81)', 'A', 'Red Sox', '(83-69)', 'Lucas Giolito', 'RHP']
['Blue Jays', '(89-63)', 'A', 'Rays', '(74-78)', 'Ian Seymour', 'LHP']
['Padres', '(83-69)', 'A', 'Mets', '(78-74)', 'David Peterson', 'LHP']
['Yankees', '(85-67)', 'A', 'Twins', '(66-86)', 'Taj Bradley', 'RHP']
['Mariners', '(83-69)', 'A', 'Royals', '(76-76)', 'Cole Ragans', 'LHP']
['Angels', '(69-83)', 'A', 'Brewers', '(93-59)', 'Brandon Woodruff', 'RHP']
['Rangers', '(79-74)', 'A', 'Astros', '(84-69)', 'Cristian Javier', 'RHP']
here105
['Marlins', '@', 'Rockies', '(72-80)'

,Win % #,Location,Moneyline Odds,Opponent Win % #,Opposing Pitcher ERA,Opposing Pitcher Hand,Runs L5,Hits L5,OPS L5,Runs L10,Hits L10,OPS L10,month_sin,month_cos
0,0.579,0,-145,0.425,3.57,0,5.8,12.8,0.9172,5.1,11.1,0.8499,-1.0,-1.836970e-16
1,0.425,1,120,0.579,3.21,1,1.6,4.4,0.4326,2.4,5.7,0.5127,-1.0,-1.836970e-16
2,0.500,0,-125,0.484,5.31,0,4.2,8.0,0.6440,5.1,8.4,0.7426,-1.0,-1.836970e-16
3,0.484,1,105,0.500,3.95,0,3.2,7.6,0.6262,3.6,8.0,0.6322,-1.0,-1.836970e-16
4,0.474,0,-125,0.373,3.54,1,4.6,8.2,0.7060,3.6,6.8,0.6327,-1.0,-1.836970e-16
5,0.373,1,105,0.474,2.91,0,5.4,8.0,0.7686,4.1,8.2,0.6905,-1.0,-1.836970e-16
6,0.500,0,-105,0.503,5.00,0,3.4,6.2,0.5812,3.2,6.5,0.5613,-1.0,-1.836970e-16
7,0.503,1,-115,0.500,3.88,0,6.4,11.0,0.8322,4.7,9.1,0.7029,-1.0,-1.836970e-16
8,0.458,0,-150,0.405,4.12,0,4.6,7.8,0.7246,5.5,8.7,0.7734,-1.0,-1.836970e-16
9,0.405,1,125,0.458,2.88,0,2.6,6.6,0.5560,2.5,5.9,0.5309,-1.0,-1.836970e-16


In [20]:
games

array(['Cubs', 'Pirates', 'Reds', 'Cardinals', 'Orioles', 'White Sox',
       'Giants', 'Diamondbacks', 'Braves', 'Nationals', 'Guardians',
       'Tigers', 'Athletics', 'Red Sox', 'Blue Jays', 'Rays', 'Padres',
       'Mets', 'Yankees', 'Twins', 'Mariners', 'Royals', 'Angels',
       'Brewers', 'Rangers', 'Astros', 'Marlins', 'Rockies', 'Phillies',
       'Dodgers'], dtype=object)

In [21]:

X_new = day_df

y_pred_new = final_lgb.predict(X_new)
print(len(games))
print(date_to_predict.strftime("%m-%d-%y"))
game_indicator = 0
for i in range(len(games)-1):
    if game_indicator == 0:
        game_indicator = 1
        print(f"Game: {games[i]} vs {games[i+1]}")
        print(f"\t{round(y_pred_new[i].item(), 3)} to {round(y_pred_new[i+1].item(), 3)}")
    else:
        game_indicator = 0


[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
30
09-17-25
Game: Cubs vs Pirates
	4.656 to 3.595
Game: Reds vs Cardinals
	5.058 to 3.992
Game: Orioles vs White Sox
	3.986 to 3.515
Game: Giants vs Diamondbacks
	4.736 to 4.093
Game: Braves vs Nationals
	4.254 to 3.42
Game: Guardians vs Tigers
	4.838 to 4.203
Game: Athletics vs Red Sox
	3.835 to 6.139
Game: Blue Jays vs Rays
	4.246 to 3.835
Game: Padres vs Mets
	4.935 to 3.653
Game: Yankees vs Twins
	5.392 to 3.747
Game: Mariners vs Royals
	5.182 to 5.207
Game: Angels vs Brewers
	3.608 to 5.402
Game: Rangers vs Astros
	4.613 to 4.023
Game: Marlins vs Rockies
	6.09 to 3.795
Game: Phillies vs Dodgers
	3.849 to 4.536


In [22]:
df_clean.size

315000

In [23]:
pick_a_date = datetime(2025, 6, 19)
pick_a_day_df = df_clean[df_clean['Date'] == pick_a_date]
pick_a_day_df


,Date,Team,Win % #,Score,Location,Moneyline Odds,Opponent,Opponent Win % #,Opposing Pitcher ERA,Opposing Pitcher Hand,...,Hits L10,OPS L10,Next Runs L5,Next Runs L10,Next Hits L5,Next Hits L10,Next OPS L5,Next OPS L10,month_sin,month_cos
208,2025-06-19,Angels,0.486,3,0,230,Yankees,0.581,3.10,1,...,8.0,0.6852,2.8,3.7,7.0,8.0,0.6710,0.7016,1.224647e-16,-1.0
652,2025-06-19,Astros,0.573,4,0,-145,Athletics,0.403,4.25,1,...,9.8,0.7669,4.0,4.8,10.4,10.0,0.7232,0.7859,1.224647e-16,-1.0
1093,2025-06-19,Athletics,0.403,6,1,120,Astros,0.573,4.54,1,...,8.2,0.6906,4.8,4.1,9.4,8.6,0.8198,0.7331,1.224647e-16,-1.0
1518,2025-06-19,Blue Jays,0.541,5,1,-160,Diamondbacks,0.500,3.88,0,...,8.6,0.6887,3.8,4.2,8.6,8.4,0.7066,0.7169,1.224647e-16,-1.0
1945,2025-06-19,Braves,0.466,7,1,-145,Mets,0.600,3.04,0,...,9.5,0.8652,7.0,6.5,10.0,9.2,0.8786,0.8303,1.224647e-16,-1.0
2375,2025-06-19,Brewers,0.533,8,0,110,Cubs,0.608,3.84,0,...,6.4,0.6430,4.4,4.3,7.0,7.2,0.6600,0.6966,1.224647e-16,-1.0
2803,2025-06-19,Cardinals,0.527,5,0,-135,White Sox,0.311,4.50,0,...,8.7,0.7261,6.4,5.4,9.4,9.1,0.8152,0.7603,1.224647e-16,-1.0
2804,2025-06-19,Cardinals,0.533,8,0,-170,White Sox,0.307,2.94,0,...,9.1,0.7603,6.4,6.0,10.2,9.4,0.8126,0.7927,1.224647e-16,-1.0
3229,2025-06-19,Cubs,0.608,7,1,-130,Brewers,0.533,2.76,0,...,7.9,0.7083,5.8,5.1,8.2,7.6,0.7500,0.7109,1.224647e-16,-1.0
3671,2025-06-19,Diamondbacks,0.500,9,0,135,Blue Jays,0.541,4.60,0,...,8.3,0.7434,5.0,5.5,8.8,8.9,0.7376,0.7986,1.224647e-16,-1.0


### predict wins

In [24]:
use_data_date = date_to_add
copy_df = pd.read_csv(f"daily_entries/daily_entries_{use_data_date.strftime("%m_%d")}.csv")

In [25]:
copied = copy_df.sort_values(['Team', 'Date'])

# 3) Compute L5 and L10 by grouping on Team:
copied['Runs L5'] = (
    copied
    .groupby('Team')['Runs']
    .transform(lambda x: x.shift(1)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)

copied['Hits L5'] = (
    copied
    .groupby('Team')['Hits']
    .transform(lambda x: x.shift(1)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)

copied['OPS L5'] = (
    copied
    .groupby('Team')['OPS']
    .transform(lambda x: x.shift(1)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)


copied['Runs L10'] = (
    copied
    .groupby('Team')['Runs']
    .transform(lambda x: x.shift(1)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

copied['Hits L10'] = (
    copied
    .groupby('Team')['Hits']
    .transform(lambda x: x.shift(1)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

copied['OPS L10'] = (
    copied
    .groupby('Team')['OPS']
    .transform(lambda x: x.shift(1)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

# 3) Compute L5 and L10 by grouping on Team:
copied['Next Runs L5'] = (
    copied
    .groupby('Team')['Runs']
    .transform(lambda x: x.shift(0)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)

copied['Next Hits L5'] = (
    copied
    .groupby('Team')['Hits']
    .transform(lambda x: x.shift(0)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)

copied['Next OPS L5'] = (
    copied
    .groupby('Team')['OPS']
    .transform(lambda x: x.shift(0)                # exclude current game
                    .rolling(window=5,
                                min_periods=1)     # use fewer games if < 5 exist
                    .mean())
)


copied['Next Runs L10'] = (
    copied
    .groupby('Team')['Runs']
    .transform(lambda x: x.shift(0)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

copied['Next Hits L10'] = (
    copied
    .groupby('Team')['Hits']
    .transform(lambda x: x.shift(0)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)

copied['Next OPS L10'] = (
    copied
    .groupby('Team')['OPS']
    .transform(lambda x: x.shift(0)
                    .rolling(window=10,
                                min_periods=1)
                    .mean())
)


df = copied[['Date', 'Team', 'Win % #', 'Score', 'Location', 'Moneyline Odds',
        'Opponent', 'Opponent Win % #',
        'Opposing Pitcher ERA', 'Opposing Pitcher Hand',
        'Runs L5', 'Hits L5', 'OPS L5',
        'Runs L10', 'Hits L10', 'OPS L10',
        'Next Runs L5', 'Next Runs L10', 'Next Hits L5',
        'Next Hits L10', 'Next OPS L5', 'Next OPS L10']]
df

numeric_cols = [
    'Win % #',
    'Score',
    'Moneyline Odds',
    'Opponent Win % #',
    'Opposing Pitcher ERA',
    'Runs L5',
    'Hits L5',
    'OPS L5',
    'Runs L10',
    'Hits L10',
    'OPS L10',
    'Next Runs L5', 'Next Runs L10', 'Next Hits L5',
    'Next Hits L10', 'Next OPS L5', 'Next OPS L10'
]

int_cols = ['Score', 'Location', 'Moneyline Odds', 'Opposing Pitcher Hand',]
float_cols = ['Win % #', 'Opponent Win % #', 'Opposing Pitcher ERA', 'Runs L5',
                'Hits L5', 'OPS L5', 'Runs L10', 'Hits L10', 'OPS L10',
                'Next Runs L5', 'Next Runs L10', 'Next Hits L5',
                'Next Hits L10', 'Next OPS L5', 'Next OPS L10']

df_clean = df.dropna(subset=numeric_cols).copy()
df_clean.reset_index(drop=True, inplace=True)

df_clean[int_cols] = df_clean[int_cols].astype(int)
df_clean[float_cols] = df_clean[float_cols].astype(float)

df_clean['Date'] = pd.to_datetime(df_clean['Date'])


df_clean['month_sin']  = np.sin(2 * np.pi * df_clean['Date'].dt.month  / 12)
df_clean['month_cos']  = np.cos(2 * np.pi * df_clean['Date'].dt.month  / 12)



In [26]:
df_map = (
    df_clean[['Date','Team','Score']]
    .rename(columns={
        'Team':'Opponent',
        'Score':'Opponent Score'
    })
)

df_wins = df_clean.merge(
    df_map,
    on=['Date','Opponent'],
    how='left'
)

df_wins["Win"] = (df_wins["Score"] > df_wins["Opponent Score"]).astype(int)

In [27]:
df_wins_red = pd.get_dummies(
    df_wins,
    columns=['Team', 'Opponent'],
    prefix=['Team', 'Opp'],
    drop_first=True
)

keep_cols_win = ['Win % #', 'Score', 'Win', 'Location', 'Moneyline Odds', 'Opponent Win % #',
            'Opposing Pitcher ERA', 'Opposing Pitcher Hand', 'Runs L5', 'Hits L5',
            'OPS L5', 'Runs L10', 'Hits L10', 'OPS L10', 'month_sin', 'month_cos']
df_wins_red = df_wins_red[keep_cols_win]
df_wins_red.size
len(df_wins_red)


13469

In [28]:
X_win = df_wins_red.drop(columns=['Score', 'Win'])

y_win = df_wins_red["Win"]

X_train_win, X_test_win, y_train_win, y_test_win = train_test_split(
    X_win, y_win, test_size=0.20, random_state=42, stratify=y_win
)


In [29]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer, roc_auc_score
import numpy as np


In [30]:
best_params_win =  {'bagging_fraction': 0.6, 'bagging_freq': 0, 'feature_fraction': 0.6, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'num_leaves': 31}

# best_params_win = {'bagging_fraction': 0.4, 'bagging_freq': 0, 'feature_fraction': 0.4, 'learning_rate': 0.005, 'max_depth': 5, 'n_estimators': 100, 'num_leaves': 31}

final_win_lgb = LGBMClassifier(**best_params_win)

final_win_lgb.fit(X_train_win, y_train_win)

[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Info] Number of positive: 5370, number of negative: 5405
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000485 seconds.
You can set `force_col_wise=true` to remove the o

LGBMClassifier(bagging_fraction=0.6, bagging_freq=0, feature_fraction=0.6,
               learning_rate=0.01, max_depth=5)

In [31]:
# pred_date = datetime(2025, 7, 1)
pred_date = date_to_predict
day_df_win, games_win, invalid_opps, spreads = get_test(pred_date, df_clean)
# day_df_win

['Cubs', '(88-64)', 'A', 'Pirates', '(65-88)', 'Johan Oviedo', 'RHP']
['Reds', '(76-76)', 'A', 'Cardinals', '(74-79)', 'Andre Pallante', 'RHP']
['Orioles', '(72-80)', 'A', 'White Sox', '(57-96)', 'Martín Pérez', 'LHP']
['Giants', '(76-76)', 'A', 'Diamondbacks', '(77-76)', 'Brandon Pfaadt', 'RHP']
['Braves', '(70-83)', 'A', 'Nationals', '(62-91)', 'Brad Lord', 'RHP']
['Guardians', '(80-71)', 'A', 'Tigers', '(85-67)', 'Jack Flaherty', 'RHP']
['Athletics', '(71-81)', 'A', 'Red Sox', '(83-69)', 'Lucas Giolito', 'RHP']
['Blue Jays', '(89-63)', 'A', 'Rays', '(74-78)', 'Ian Seymour', 'LHP']
['Padres', '(83-69)', 'A', 'Mets', '(78-74)', 'David Peterson', 'LHP']
['Yankees', '(85-67)', 'A', 'Twins', '(66-86)', 'Taj Bradley', 'RHP']
['Mariners', '(83-69)', 'A', 'Royals', '(76-76)', 'Cole Ragans', 'LHP']
['Angels', '(69-83)', 'A', 'Brewers', '(93-59)', 'Brandon Woodruff', 'RHP']
['Rangers', '(79-74)', 'A', 'Astros', '(84-69)', 'Cristian Javier', 'RHP']
here105
['Marlins', '@', 'Rockies', '(72-80)'

## score diff based on odds

In [32]:
import math
from math import exp, factorial, sqrt
from mpmath import besseli

# Poisson CDF: P(T <= k)
def pois_cdf(k, lam):
    return exp(-lam) * sum(lam**i / factorial(i) for i in range(k+1))

def infer_S_from_total(total_line, p_over, p_under):
    # map "Over x.5" -> P(T >= ceil(x.5)) = p_over
    k = math.floor(total_line) + 1  # e.g., 7.5 -> k=8

    # binary search for S
    lo, hi = 0.1, 20.0
    for _ in range(60):
        mid = (lo + hi) / 2
        p_tail = 1 - pois_cdf(k-1, mid)  # P(T >= k)
        if p_tail < p_over:
            lo = mid
        else:
            hi = mid
    return 0.5 * (lo + hi)

def skellam_pmf(k, l1, l2):
    return exp(-(l1+l2)) * (l1/l2)**(k/2) * float(besseli(abs(k), 2*sqrt(l1*l2)))

def P_D_gt_0(mu, S):
    l1, l2 = (S+mu)/2, (S-mu)/2
    s, k = 0.0, 1
    while True:
        t = skellam_pmf(k, l1, l2)
        s += t
        if t < 1e-12 and k > 50: break
        k += 1
    return s

def devig_odds(odds1, odds2):
    odds1_num = int(odds1)
    odds1_exp = -odds1_num / (-odds1_num+100) if odds1_num < 0 else 100/(odds1_num+100)
    odds2_num = int(odds2)
    odds2_exp = -odds2_num / (-odds2_num+100) if odds2_num < 0 else 100/(odds2_num+100)

    odds1_revig = round(odds1_exp / (odds1_exp+odds2_exp), 3)
    odds2_revig = round(odds2_exp / (odds1_exp+odds2_exp), 3)

    return odds1_revig, odds2_revig

def expected_scores(away_odds, home_odds):
    # 0: team
    # 1: ml
    # 2: spread
    # 3: spread odds
    # 4: total line
    # 5: over total odds
    # 6: under total odds

    away_devig, home_devig = devig_odds(away_odds[1], home_odds[1])
    fav_devig = away_devig if away_devig > home_devig else home_devig

    over_devig, under_devig = devig_odds(away_odds[5], away_odds[6])

    S = infer_S_from_total(float(away_odds[4]), over_devig, under_devig)

    lo, hi = -S + 1e-6, S - 1e-6        # mu bounds
    for _ in range(60):                  # bisection on mu
        mid = (lo + hi) / 2
        if P_D_gt_0(mid, S) < fav_devig:
            lo = mid
        else:
            hi = mid
    mu = (lo + hi) / 2

    if fav_devig == away_devig:
        return round((S + mu)/2, 3), round((S - mu)/2, 3), round(mu, 3)
    elif fav_devig == home_devig:
        return round((S - mu)/2, 3), round((S + mu)/2, 3), round(-mu, 3)

# predictions

### score and wins

In [33]:

X_new = day_df

y_pred_new = final_lgb.predict(X_new)
print(len(games))
print(date_to_predict.strftime("%m-%d-%y"))
for i in range(0, len(games)-1, 2):
    away = games[i]
    home = games[i+1]
    away_pred = round(y_pred_new[i].item(), 3)
    home_pred = round(y_pred_new[i+1].item(), 3)
    # print(f"Game: {away} @ {home}")
    # print(f"\t{away_pred} to {home_pred}")
    away_spread = spreads[i]
    home_spread = spreads[i+1]

    winner = home if home_pred >= away_pred else away
    # hm, am = _ml_lookup.get((home, away), (None, None))
    # ml = hm if winner == home else am
    if away in invalid_opp_pitchers or home in invalid_opp_pitchers:
        print(f"Pitchers TBD ")
    if winner == home:
        # print(f"Winner: {winner}, Score: {away_pred} to {home_pred} (ML: {day_df['Moneyline Odds'][i+1]})")
        print(f"Game: {away} @ {home}, {away_pred} to {home_pred} (ML: {day_df['Moneyline Odds'][i+1]})")
    elif winner == away:
        # print(f"Winner: {winner}, Score: {away_pred} to {home_pred} (ML: {day_df['Moneyline Odds'][i]})")
        print(f"Game: {away} @ {home}, {away_pred} to {home_pred} (ML: {day_df['Moneyline Odds'][i]})")
    else:
        print(f"Winner: {winner} (ML: N/A)")
    
    print(expected_scores(away_spread, home_spread))
    # print(home_spread)
    print("__________________")




[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
30
09-17-25
Game: Cubs @ Pirates, 4.656 to 3.595 (ML: -145)
(4.36, 3.386, 0.974)
__________________
Game: Reds @ Cardinals, 5.058 to 3.992 (ML: -125)
(4.667, 3.921, 0.746)
__________________
Game: Orioles @ White Sox, 3.986 to 3.515 (ML: -125)
(4.626, 3.882, 0.745)
__________________
Game: Giants @ Diamondbacks, 4.736 to 4.093 (ML: -105)
(4.237, 4.829, -0.593)
__________________
Game: Braves @ Nationals, 4.254 to 3.42 (ML: -150)
(4.822, 3.766, 1.057)
__________________
Game: Guardians @ Tigers, 4.838 to 4.203 (ML: 120)
(3.457, 4.435, -0.978)
__________________
Game: Athletics @ Red Sox, 3.835 to 6.139 (ML: -165)
(4.032, 5.306, -1.275)
________

In [34]:
X_new_win = day_df_win

y_pred_new_win = final_win_lgb.predict(X_new_win)
y_proba_win = final_win_lgb.predict_proba(X_new_win)
print(pred_date.strftime("%m-%d-%y"))
for i in range(0, len(games_win)-1, 2):
    away_win = games_win[i]
    home_win = games_win[i+1]
    away_prob = (y_proba_win[i][1] + y_proba_win[i+1][0]) / 2
    home_prob = (y_proba_win[i][0] + y_proba_win[i+1][1]) / 2
    # print(f"Game: {away_win} @ {home_win}")
    # team1_prob = (y_proba_win[i][1] + y_proba_win[i+1][0]) / 2
    # team2_prob = (y_proba_win[i][0] + y_proba_win[i+1][1]) / 2
    if away_prob > home_prob:
        print(f"Winner: {away_win}, Conf: {round(away_prob*100, 2)} (ML: {day_df_win['Moneyline Odds'][i]})")
    elif away_prob < home_prob:
        print(f"Winner: {home_win}, Conf: {round(home_prob*100, 2)} (ML: {day_df_win['Moneyline Odds'][i+1]})")
    else:
        print("uhoh")
    print("__________________")


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
09-17-25
Winner: Cubs, Conf: 56.03 (ML: -145)
__________________
Winner: Reds, Conf: 51.88 (ML: -125)
__________________
Winner: Orioles, Conf: 53.01 (ML: -125)
__________________
Winner: Giants, Conf: 51.12 (ML: -105)
__________________
Winner: Braves, Conf: 56.21 (ML: -150)
__________________
Winner

## all predictions

In [35]:
from termcolor import colored
# run predictions: y_pred_new
# win predictions: y_proba_win
bets = []
this_date = pred_date.strftime("%m-%d-%y")
print(this_date)
print(f"{int(len(games)/2)} games")
for i in range(0, len(games)-1, 2):
    # teams
    away = games[i]
    home = games[i+1]

    ## odds
    # 0: team
    # 1: ml
    # 2: spread
    # 3: spread odds
    # 4: total line
    # 5: over total odds
    # 6: under total odds
    away_spread = spreads[i]
    home_spread = spreads[i+1]

    # score predictions
    away_score_pred = round(y_pred_new[i].item(), 3)
    home_score_pred = round(y_pred_new[i+1].item(), 3)

    # win probability predictions
    away_win_prob = round(((y_proba_win[i][1] + y_proba_win[i+1][0]) / 2) * 100, 1)
    home_win_prob = round(((y_proba_win[i][0] + y_proba_win[i+1][1]) / 2)  * 100, 1)

    # expected scores
    away_exp_score, home_exp_score, exp_score_diff = expected_scores(away_spread, home_spread)

    # expected win probability
    away_exp_win, home_exp_win = devig_odds(away_spread[1], home_spread[1])
    away_exp_win = round(away_exp_win * 100, 3)
    home_exp_win = round(home_exp_win * 100, 3)

    # calc diffs and totals
    predicted_diff = round(away_score_pred-home_score_pred, 3)
    # exp_diff = round(away_exp_score-home_exp_score, 3)
    predicted_total = round(away_score_pred+home_score_pred, 3)
    exp_total = round(away_exp_score+home_exp_score, 3)
    #{colored(era[1], 'green')}
    print(f"\t\t{away} @ {home}")
    if away in invalid_opp_pitchers or home in invalid_opp_pitchers:
        print(f"Pitchers TBD ")
        print("________________________________________________________________________________")
        continue

    print(f"predicted scores:\t\t{away_score_pred} to {home_score_pred}")
    print(f"exp scores:\t\t\t{away_exp_score} to {home_exp_score}")
    # diffs
    if abs(predicted_diff) > abs(exp_score_diff):
        print(f"predicted vs exp diff:\t\t{predicted_diff} vs {exp_score_diff}\t\t{colored(away if predicted_diff > 0  else home, 'green')} {colored(away_spread[1] if predicted_diff > 0  else home_spread[1], 'green')} {round(predicted_diff - exp_score_diff, 3)}")
        bets.append([this_date, away, home, away if predicted_diff > 0  else home, away_spread[1] if predicted_diff > 0  else home_spread[1], abs(predicted_diff), abs(round(predicted_diff - exp_score_diff, 3)), '', '', '', ''])
    else:
        print(f"predicted vs exp diff:\t\t{predicted_diff} vs {exp_score_diff}")
    
    # win probs
    if away_win_prob > home_win_prob: # away predicted to win
        print(f"\t\t\t{away} win")
        if away_win_prob > away_exp_win:
            print(f"predicted vs exp win %:\t\t{away_win_prob} % vs {away_exp_win} %\t{colored(away, 'green')} {colored(away_spread[1], 'green')} {round(away_win_prob - away_exp_win, 3)} %")
            if len(bets) >= 1:
                if bets[-1][1:4] == [away, home, away]:
                    bets[-1][7] = away_win_prob
                    bets[-1][8] = round(away_win_prob - away_exp_win, 3)
                else:
                    bets.append([this_date, away, home, away, away_spread[1], '', '', away_win_prob, round(away_win_prob - away_exp_win, 3), '', ''])
            else:
                bets.append([this_date, away, home, away, away_spread[1], '', '', away_win_prob, round(away_win_prob - away_exp_win, 3), '', ''])
        else:
            print(f"predicted vs exp win %:\t\t{away_win_prob} % vs {away_exp_win} %")
    elif away_win_prob < home_win_prob: # home team predicted to win
        print(f"\t\t\t{home} win")
        if home_win_prob > home_exp_win:
            print(f"predicted vs exp win %:\t\t{home_win_prob} % vs {home_exp_win} %\t{colored(home, 'green')} {colored(home_spread[1], 'green')} {round(home_win_prob - home_exp_win, 3)} %")
            if len(bets) >= 1:
                if bets[-1][1:4] == [away, home, home]:
                    bets[-1][7] = home_win_prob
                    bets[-1][8] = round(home_win_prob - home_exp_win, 3)
                else:
                    bets.append([this_date, away, home, home, home_spread[1], '', '', home_win_prob, round(home_win_prob - home_exp_win, 3), '', ''])
            else:
                bets.append([this_date, away, home, home, home_spread[1], '', '', home_win_prob, round(home_win_prob - home_exp_win, 3), '', ''])
        else:
            print(f"predicted vs exp win %:\t\t{home_win_prob} % vs {home_exp_win} %")
    else:
        print(f"away predicted vs exp win %:\t\t{away_win_prob} % vs {away_exp_win} %")
        print(f"home predicted vs exp win %:\t\t{home_win_prob} % vs {home_exp_win} %")

    # totals
    if predicted_total - exp_total > 1:
        print(f"predicted vs expected total:\t{predicted_total} vs {exp_total}\t\t{colored("Over", 'green')} {colored(away_spread[4], 'green')} {colored(away_spread[5], 'green')} {round(predicted_total - exp_total, 3)}")
        bets.append([this_date, away, home, f"Over {away_spread[4]}", away_spread[5], '', '', '', '',  round(predicted_total - exp_total, 3), ''])
    elif predicted_total - exp_total < -1:
        print(f"predicted vs expected total:\t{predicted_total} vs {exp_total}\t\t{colored("Under", 'green')} {colored(away_spread[4], 'green')} {colored(away_spread[6], 'green')} {round(predicted_total - exp_total, 3)}")
        bets.append([this_date, away, home, f"Under {away_spread[4]}", away_spread[6], '', '', '', '', round(predicted_total - exp_total, 3), ''])
    else:
        print(f"predicted vs expected total:\t{predicted_total} vs {exp_total}")

    print("________________________________________________________________________________")

bet_data = pd.DataFrame(bets, columns=['Date', 'Away Team', 'Home Team', 'Pick', 'Odds', 'Projected Score Diff', 'Score Diff from Expected', 'Projected Win %', 'Win % Diff from Expected', 'Total Diff', 'Result'])
bet_data


09-17-25
15 games
		Cubs @ Pirates
predicted scores:		4.656 to 3.595
exp scores:			4.36 to 3.386
predicted vs exp diff:		1.061 vs 0.974		Cubs -145 0.087
			Cubs win
predicted vs exp win %:		56.0 % vs 56.6 %
predicted vs expected total:	8.251 vs 7.746
________________________________________________________________________________
		Reds @ Cardinals
predicted scores:		5.058 to 3.992
exp scores:			4.667 to 3.921
predicted vs exp diff:		1.066 vs 0.746		Reds -125 0.32
			Reds win
predicted vs exp win %:		51.9 % vs 53.2 %
predicted vs expected total:	9.05 vs 8.588
________________________________________________________________________________
		Orioles @ White Sox
predicted scores:		3.986 to 3.515
exp scores:			4.626 to 3.882
predicted vs exp diff:		0.471 vs 0.745
			Orioles win
predicted vs exp win %:		53.0 % vs 53.2 %
predicted vs expected total:	7.501 vs 8.508		Under 8.5 -120 -1.007
________________________________________________________________________________
		Giants @ Diamondbacks


,Date,Away Team,Home Team,Pick,Odds,Projected Score Diff,Score Diff from Expected,Projected Win %,Win % Diff from Expected,Total Diff,Result
0,09-17-25,Cubs,Pirates,Cubs,-145,1.061,0.087,,,,
1,09-17-25,Reds,Cardinals,Reds,-125,1.066,0.32,,,,
2,09-17-25,Orioles,White Sox,Under 8.5,-120,,,,,-1.007,
3,09-17-25,Giants,Diamondbacks,Giants,-105,0.643,1.236,51.1,2.2,,
4,09-17-25,Guardians,Tigers,Over 7.5,-125,,,,,1.149,
5,09-17-25,Athletics,Red Sox,Red Sox,-165,2.304,1.029,,,,
6,09-17-25,Padres,Mets,Padres,+115,1.282,2.158,51.8,7.1,,
7,09-17-25,Padres,Mets,Over 7.5,+105,,,,,1.138,
8,09-17-25,Yankees,Twins,Yankees,-155,1.645,0.49,,,,
9,09-17-25,Mariners,Royals,Mariners,+105,,,50.3,3.5,,


### put data into archive

In [36]:
# archive = pd.read_csv("predictions.csv")
# # del archive['Unnamed: 0']
# yesterday_date = date_to_add.strftime("%m-%d-%y")
# yesterday = archive[archive['Date'] == yesterday_date]
# yesterday = yesterday.reset_index(drop=True)
# print(doubles)
# for i in range(len(yesterday)):
#     game = yesterday.iloc[i].to_dict()
#     # print(game)
#     away_score = int(yesterday_entries.loc[yesterday_entries['Team'] == yesterday.at[i, 'Away Team'], 'Score'].iloc[0])
#     home_score = int(yesterday_entries.loc[yesterday_entries['Team'] == yesterday.at[i, 'Home Team'], 'Score'].iloc[0])
#     # print(f"{away_score} to {home_score}")
    
#     if game['Pick'] == game['Away Team']:
#         if away_score > home_score:
#             archive.loc[(archive['Date'] == yesterday_date) &
#                 (archive['Away Team'] == yesterday.at[i, 'Away Team']) &
#                 (archive['Home Team'] == yesterday.at[i, 'Home Team']) &
#                 (archive['Pick'] == yesterday.at[i, 'Away Team']),
#                 'Result'] = 'Win'
#         else:
#             archive.loc[(archive['Date'] == yesterday_date) &
#                 (archive['Away Team'] == yesterday.at[i, 'Away Team']) &
#                 (archive['Home Team'] == yesterday.at[i, 'Home Team']) &
#                 (archive['Pick'] == yesterday.at[i, 'Away Team']),
#                 'Result'] = 'Loss'
#     elif game['Pick'] == game['Home Team']:
#         if away_score < home_score:
#             archive.loc[(archive['Date'] == yesterday_date) &
#                 (archive['Away Team'] == yesterday.at[i, 'Away Team']) &
#                 (archive['Home Team'] == yesterday.at[i, 'Home Team']) &
#                 (archive['Pick'] == yesterday.at[i, 'Home Team']),
#                 'Result'] = 'Win'
#         else:
#             archive.loc[(archive['Date'] == yesterday_date) &
#                 (archive['Away Team'] == yesterday.at[i, 'Away Team']) &
#                 (archive['Home Team'] == yesterday.at[i, 'Home Team']) &
#                 (archive['Pick'] == yesterday.at[i, 'Home Team']),
#                 'Result'] = 'Loss'
#     elif game['Pick'].split()[0] == 'Over':
#         if int(away_score) + int(home_score) > float(game['Pick'].split()[1]):
#             archive.loc[(archive['Date'] == yesterday_date) &
#                 (archive['Away Team'] == yesterday.at[i, 'Away Team']) &
#                 (archive['Home Team'] == yesterday.at[i, 'Home Team']) &
#                 (archive['Pick'].str.split(n=1).str[0] == 'Over'),
#                 'Result'] = 'Win'
#         else:
#             archive.loc[(archive['Date'] == yesterday_date) &
#                 (archive['Away Team'] == yesterday.at[i, 'Away Team']) &
#                 (archive['Home Team'] == yesterday.at[i, 'Home Team']) &
#                 (archive['Pick'].str.split(n=1).str[0] == 'Over'),
#                 'Result'] = 'Loss'
#     elif game['Pick'].split()[0] == 'Under':
#         if int(away_score) + int(home_score) < float(game['Pick'].split()[1]):
#             archive.loc[(archive['Date'] == yesterday_date) &
#                 (archive['Away Team'] == yesterday.at[i, 'Away Team']) &
#                 (archive['Home Team'] == yesterday.at[i, 'Home Team']) &
#                 (archive['Pick'].str.split(n=1).str[0] == 'Under'),
#                 'Result'] = 'Win'
#         else:
#             archive.loc[(archive['Date'] == yesterday_date) &
#                 (archive['Away Team'] == yesterday.at[i, 'Away Team']) &
#                 (archive['Home Team'] == yesterday.at[i, 'Home Team']) &
#                 (archive['Pick'].str.split(n=1).str[0] == 'Under'),
#                 'Result'] = 'Loss'
#     else:
#         print("oops")
#     # print(yesterday.iloc[i].to_dict())
#     # print("__________________________________")
# archive.tail(len(yesterday))

In [37]:
# a_w_pred = pd.concat([archive, bet_data])
# a_w_pred.set_index('Date', inplace=True)
# a_w_pred.to_csv("predictions.csv")
# # a_w_pred
# # bet_data

### pick a date

In [38]:
pick_a_date = datetime(2025, 7, 19)
pick_a_day_df = df_clean[df_clean['Date'] == pick_a_date]
# pick_a_day_df = get_for_date(my_date)
pick_a_day_df


,Date,Team,Win % #,Score,Location,Moneyline Odds,Opponent,Opponent Win % #,Opposing Pitcher ERA,Opposing Pitcher Hand,...,Hits L10,OPS L10,Next Runs L5,Next Runs L10,Next Hits L5,Next Hits L10,Next OPS L5,Next OPS L10,month_sin,month_cos
277,2025-07-19,Angels,0.490,5,0,110,Phillies,0.571,3.44,0,...,8.1,0.7641,5.2,5.9,8.4,9.0,0.8332,0.8210,-0.5,-0.866025
719,2025-07-19,Astros,0.571,6,0,120,Mariners,0.541,4.37,0,...,8.9,0.6557,3.4,4.8,7.6,9.2,0.5574,0.6747,-0.5,-0.866025
1157,2025-07-19,Athletics,0.420,8,0,115,Guardians,0.485,4.35,1,...,8.8,0.8875,7.2,6.9,10.8,9.4,1.0180,0.9131,-0.5,-0.866025
1583,2025-07-19,Blue Jays,0.582,6,1,105,Giants,0.525,3.13,0,...,8.8,0.7224,3.0,4.5,8.6,9.7,0.6666,0.7666,-0.5,-0.866025
2009,2025-07-19,Braves,0.443,9,1,110,Yankees,0.551,4.47,0,...,8.1,0.7535,6.6,5.7,7.8,8.1,0.7694,0.7535,-0.5,-0.866025
2439,2025-07-19,Brewers,0.592,8,0,105,Dodgers,0.586,4.17,0,...,8.1,0.7075,4.4,4.8,9.0,8.5,0.6824,0.7342,-0.5,-0.866025
2866,2025-07-19,Cardinals,0.515,1,0,-135,Diamondbacks,0.495,3.63,0,...,10.1,0.8253,4.2,5.1,9.0,9.7,0.7312,0.7851,-0.5,-0.866025
3293,2025-07-19,Cubs,0.602,6,1,-160,Red Sox,0.530,3.07,0,...,10.1,0.8536,7.4,6.8,11.2,10.2,0.9260,0.8841,-0.5,-0.866025
3740,2025-07-19,Diamondbacks,0.495,10,1,115,Cardinals,0.515,4.33,0,...,10.2,0.7799,8.6,6.4,12.2,10.0,0.9030,0.7769,-0.5,-0.866025
4181,2025-07-19,Dodgers,0.586,7,1,-125,Brewers,0.592,2.68,0,...,7.3,0.6226,5.2,4.7,7.4,7.7,0.6564,0.6603,-0.5,-0.866025
